# Lab 3: Global Average Sea Surface Salinity Map on ArcGIS Online
### Liliana Valle
### December 11, 2025
#### This script calculates the mean salinity across all monthly observations (2011-2024), prepares, and publishes the resulting layer to ArcGIS Online.



## Summary: Total Average Salinity Mapping

This lab creates a web accessible visualization of global sea surface salinity patterns by publishing a 14 year average map to ArcGIS Online. The workflow begins with calculating the mean salinity across 160 monthly NASA satellite observations from 2011-2024 using Python geoprocessing. The resulting global average highlights persistent oceanographic features: higher salinity regions in subtropical gyres, and lower salinity zones near major river mouths and in polar regions.

The raster data were optimized for web display through tiling/classification and exported as a Cloud Optimized GeoTIFF. A Python pipeline demonstrated the upload process to ArcGIS Online, including metadata tagging and tile layer publication. The final web map employs a color scheme with lighter tones representing lower salinity regions and darker showcasing higher salinity regions.This web publication makes specialized oceanographic data accessible to broader audiences, supporting educational and preliminary research applications. 

I have kept this original code in the notebook (that was not working for me), even though I ended up manually uploading many maps to my content in AGOL. I first uploaded TIFF (Global Average Salinity 2011-2024), however that will not be used in the final version of this project. I was also able to upload the same map as a web map (Global_Avg_Story) and a hosted tile layer (Global_Total_Avg_Salinity_Story).

## Section 1: Setup and Using cell stats to calculate global mean

In [26]:
import arcpy
import os
from arcpy.sa import *

# Setup
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"C:\Users\Liliana\Documents\ArcGIS\Projects\SalinityDrafting\Salinity_Analysis.gdb"

# Get all processed rasters
all_rasters = arcpy.ListRasters("SSS_20*")
print(f"Found {len(all_rasters)} monthly salinity rasters")


Found 160 monthly salinity rasters


In [30]:
# Calculate global mean
print("Calculating global mean across all months...")
global_mean = CellStatistics(all_rasters, "MEAN", "DATA")

# Save the result
output_name = "Global_Mean_Salinity_2011_2024"
global_mean.save(output_name)
print(f"✓ Global mean saved as: {output_name}")

Calculating global mean across all months...
✓ Global mean saved as: Global_Mean_Salinity_2011_2024


#### Add metadata for AGOL

In [29]:
metadata = """
<title>Global Mean Sea Surface Salinity (2011-2024)</title>
<abstract>Average sea surface salinity calculated from NASA Multi-Mission 
Optimally Interpolated monthly data spanning 2011 to 2024. This represents 
the baseline salinity conditions of the world's oceans over a 14 year period.</abstract>
<tags>salinity, oceanography, NASA, climate</tags>
"""

## Section 2: Data Prep for AGOL Publishing
Optimizing the raster for web display

In [31]:
# Reclassifying to reduce file size for web display and creating a simplified version with fewer classes
simplified_output = "Global_Mean_Salinity_Web"
reclass_ranges = arcpy.sa.RemapRange([
    [0, 20, 1],    # Very low salinity
    [20, 30, 2],   # Low salinity  
    [30, 33, 3],   # Moderate salinity
    [33, 35, 4],   # Normal ocean salinity
    [35, 38, 5],   # High salinity
    [38, 50, 6]    # Very high salinity
])
simplified_raster = arcpy.sa.Reclassify(global_mean, "Value", reclass_ranges)
simplified_raster.save(simplified_output)
print(f"✓ Simplified raster created: {simplified_output}")

✓ Simplified raster created: Global_Mean_Salinity_Web


In [32]:
# Export to Cloud Optimized GeoTIFF
print("Exporting to Cloud Optimized GeoTIFF...")
output_tiff = r"C:\Users\Liliana\Documents\ArcGIS\Projects\SalinityDrafting\Global_Mean_Salinity.tif"
arcpy.management.CopyRaster(
    simplified_output,
    output_tiff,
    config_keyword="CloudOptimized=YES"
)
print(f"✓ Cloud Optimized GeoTIFF created: {output_tiff}")

Exporting to Cloud Optimized GeoTIFF...
✓ Cloud Optimized GeoTIFF created: C:\Users\Liliana\Documents\ArcGIS\Projects\SalinityDrafting\Global_Mean_Salinity.tif


## Section 3: AGOL Upload
This automates the upload to AGOL using the ArcGIS API for Python.

In [33]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
    
# Connect to AGOL
print("Connecting to ArcGIS Online...")
gis = GIS("https://www.arcgis.com", "lvalle14", "Basalt00!")
print(f"✓ Connected as: {gis.properties.user.username}")
    

Connecting to ArcGIS Online...
✓ Connected as: lvalle14


In [44]:
# Using new Folder.add() method 

from arcgis.gis import GIS
import os

# file path
output_tiff = r"C:\Users\Liliana\Documents\ArcGIS\Projects\SalinityDrafting\Global_Mean_Salinity.tif"

# Check file exists
if not os.path.exists(output_tiff):
    print(f"File not found: {output_tiff}")
else:
    print(f"File ready: {os.path.basename(output_tiff)}")
  

File ready: Global_Mean_Salinity.tif


In [50]:
# Get user folder
user = gis.users.get(gis.properties.user.username)
folder_name = "SalinityDrafting" 

# add item to the folder
print("\nUploading file...")
item = gis.content.add(
    item_properties={
        "title": "Global Mean Salinity 2011-2024",
        "tags": "salinity, NASA, ocean"
    },
    data=output_tiff,
    folder=folder_name 
)


Uploading file...


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\IPython\core\interactiveshell.py:3550: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'RuntimeError'>: Specify type in item_properties

In [ ]:
    print(f"Uploaded! Item: {item.id}")
    
    # Publish
    print("Publishing...")
    published = item.publish()
    
    # Share
    published.share(everyone=True)
    print("Shared publicly!")
    
    print(f"View at: {published.homepage}")

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\IPython\core\interactiveshell.py:3550: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'RuntimeError'>: Specify type in item_properties